# Iceberg

## Generer testdata

In [1]:
import pyspark
import faker

fake = faker.Faker()

data = [(fake.name(), fake.address(), fake.email(), fake.phone_number()) for _ in range(1000)]

## Start Spark session
Sett opp en spark session med Iceberg konfigurert. Oppretter katalogen `iceberg_catalog`, som bruker et lokalt filsystem (`hadoop`) i mappen `./warehouse`
- Kunne også brukt kataloger basert på `jdbc`, `rest` eller `project Nessie`


In [ ]:
spark = (
    pyspark.sql.SparkSession.builder.appName("example")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg_catalog.type", "hadoop")
    .config("spark.sql.catalog.iceberg_catalog.warehouse", "../warehouse")
    .config("spark.defaultCatalog", "iceberg_catalog")
    .getOrCreate()
)

In [ ]:
spark.sql("create database if not exists iceberg_db")
df = spark.createDataFrame(data, ["name", "address", "email", "phone_number"]).coalesce(1)
df.writeTo("iceberg_catalog.iceberg_db.user").using("iceberg").createOrReplace()  

In [ ]:
spark.sql("select * from iceberg_catalog.iceberg_db.user limit 10").show()

In [6]:
spark.stop()